<a href="https://colab.research.google.com/github/singhbhupender1/ML-notebooks/blob/master/Twitter_Scraper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Twitter Scraper Online

This is an online version of the jonbakerfish/TweetScraper repository. It is one of the most complete Twitter scrapers.

Credits: Jon https://github.com/jonbakerfish/TweetScraper

Settings

In [0]:
search = '@newsbreakapp' #comma separated keywords
db = 'mongo' #mongo/mysql
download_file = True
store_in_drive = False
clean_db_before_store = True

In [0]:
if store_in_drive:
    from google.colab import drive
    drive.mount('/content/drive')

## Setup

### Scraper Setup

Verify first time

In [0]:
is_installed = !scrapy list
if is_installed[0] == 'TweetScraper':
  first_time = False
else:
  first_time = True

Install scraper

In [0]:
if first_time:
  if store_in_drive:
    from google.colab import drive
    drive.mount('/content/drive')
  !git clone https://github.com/jonbakerfish/TweetScraper.git
  %cd TweetScraper
  !pip install -r requirements.txt  #add '--user' if you are not root
  !scrapy list

Cloning into 'TweetScraper'...
remote: Enumerating objects: 30, done.
remote: Counting objects: 100% (30/30), done.
remote: Compressing objects: 100% (25/25), done.
remote: Total 256 (delta 14), reused 14 (delta 4), pack-reused 226
Receiving objects: 100% (256/256), 55.60 KiB | 307.00 KiB/s, done.
Resolving deltas: 100% (135/135), done.
/content/TweetScraper
     |████████████████████████████████| 235kB 2.8MB/s 
     |████████████████████████████████| 11.9MB 22.4MB/s 
     |████████████████████████████████| 61kB 25.5MB/s 
     |████████████████████████████████| 3.1MB 27.3MB/s 
     |████████████████████████████████| 2.3MB 32.7MB/s 
     |████████████████████████████████| 61kB 26.7MB/s 
     |████████████████████████████████| 174kB 51.3MB/s 
     |████████████████████████████████| 112kB 44.2MB/s 
  Created wheel for mysql-connector: filename=mysql_connector-2.2.9-cp36-cp36m-linux_x86_64.whl size=247949 sha256=18810f4bbb06e78bf0a887a9f7406d6c33f06198f3bbb12de68b5eacb1bdf1d9
  Stored in d

### MongoDB Setup

In [0]:
if first_time and db == 'mongo':
  !apt install mongodb
  !service mongodb start
  !sed -i "s/TweetScraper.pipelines.SaveToFilePipeline/TweetScraper.pipelines.SaveToMongoPipeline/" /content/TweetScraper/TweetScraper/settings.py

Clean the database

In [0]:
if not first_time and clean_db_before_store and db == 'mongo':
  !mongo --eval "print(db.getSiblingDB('TweetScraper').getCollection('tweet').count());print(db.getSiblingDB('TweetScraper').getCollection('user').count());"
  !mongo --eval "db.getSiblingDB('TweetScraper').getCollection('tweet').remove({});db.getSiblingDB('TweetScraper').getCollection('user').remove({})"
  !mongo --eval "print(db.getSiblingDB('TweetScraper').getCollection('tweet').count());print(db.getSiblingDB('TweetScraper').getCollection('user').count());"

### MySQL Setup

Install MySQL

In [0]:
if first_time and db == 'mysql':
  !apt install mysql-server
  !service mysql start
  !mysql -u root -e "create database TweetScraper";
  !sed -i "s/TweetScraper.pipelines.SaveToFilePipeline/TweetScraper.pipelines.SavetoMySQLPipeline/" /content/TweetScraper/TweetScraper/settings.py

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  libcgi-fast-perl libcgi-pm-perl libencode-locale-perl libevent-core-2.1-6
  libfcgi-perl libhtml-parser-perl libhtml-tagset-perl libhtml-template-perl
  libhttp-date-perl libhttp-message-perl libio-html-perl
  liblwp-mediatypes-perl libtimedate-perl liburi-perl mysql-client-5.7
  mysql-client-core-5.7 mysql-server-5.7 mysql-server-core-5.7 psmisc
Suggested packages:
  libdata-dump-perl libipc-sharedcache-perl libwww-perl mailx tinyca
The following NEW packages will be installed:
  libcgi-fast-perl libcgi-pm-perl libencode-locale-perl libevent-core-2.1-6
  libfcgi-perl libhtml-parser-perl libhtml-tagset-perl libhtml-template-perl
  libhttp-date-perl libhttp-message-perl libio-html-perl
  liblwp-mediatypes-perl libtimedate-perl liburi-perl mysql-client-5.7
  mysql-client-core-5.7 mysql-server mysql-server-5.7 mysql-server-core-5.7
  psmisc

Show the default credentials

In [0]:
if first_time and db == 'mysql':
  !cat /etc/mysql/debian.cnf

# Automatically generated for Debian scripts. DO NOT TOUCH!
[client]
host     = localhost
user     = debian-sys-maint
password = us5TMM6BOSoz4oTW
socket   = /var/run/mysqld/mysqld.sock
[mysql_upgrade]
host     = localhost
user     = debian-sys-maint
password = us5TMM6BOSoz4oTW
socket   = /var/run/mysqld/mysqld.sock


In [0]:
if first_time and db == 'mysql':
  import configparser
  config = configparser.ConfigParser()
  config.read('/etc/mysql/debian.cnf')
  db_user = config['client']['user']
  db_pass = config['client']['password']
  print(db_user)
  print(db_pass)

debian-sys-maint
us5TMM6BOSoz4oTW


Set the previously displayed credentials

In [0]:
if first_time and db == 'mysql':
  !sed -i 's/MYSQL_USER   = .*/MYSQL_USER   = "{db_user}"/' /content/TweetScraper/TweetScraper/settings.py
  !sed -i 's/MYSQL_PWD    = .*/MYSQL_PWD    = "{db_pass}"/' /content/TweetScraper/TweetScraper/settings.py

Clean the database

In [0]:
if not first_time and clean_db_before_store and db == 'mysql':
  !mysql TweetScraper -e "DELETE * from scraper"

# Run the Scraper

In [0]:
!scrapy crawl TweetScraper -a query="{search}"

2019-10-16 02:32:21 [scrapy.utils.log] INFO: Scrapy 1.7.3 started (bot: TweetScraper)
2019-10-16 02:32:21 [scrapy.utils.log] INFO: Versions: lxml 4.2.6.0, libxml2 2.9.8, cssselect 1.1.0, parsel 1.5.2, w3lib 1.21.0, Twisted 19.7.0, Python 3.6.8 (default, Jan 14 2019, 11:02:34) - [GCC 8.0.1 20180414 (experimental) [trunk revision 259383]], pyOpenSSL 19.0.0 (OpenSSL 1.1.1c  28 May 2019), cryptography 2.7, Platform Linux-4.14.137+-x86_64-with-Ubuntu-18.04-bionic
2019-10-16 02:32:21 [scrapy.crawler] INFO: Overridden settings: {'BOT_NAME': 'TweetScraper', 'LOG_LEVEL': 'INFO', 'NEWSPIDER_MODULE': 'TweetScraper.spiders', 'SPIDER_MODULES': ['TweetScraper.spiders'], 'USER_AGENT': 'TweetScraper'}
2019-10-16 02:32:21 [scrapy.extensions.telnet] INFO: Telnet Password: 49dcae699560c09e
2019-10-16 02:32:21 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.memusage.MemoryUsage',
 'scrapy.extensions.logs

# Store the data

## MongoDB save to CSV (Optional)

Get header names

In [0]:
if first_time and db == 'mongo':
  !mongo --eval "var row = db.getSiblingDB('TweetScraper').getCollection('tweet').findOne();for (var i in row) {print(i);}" > output.txt
  !tail -n +4 output.txt > fields.txt

Number of records

In [0]:
if db == 'mongo':
  !mongo --eval "print(db.getSiblingDB('TweetScraper').getCollection('tweet').count());print(db.getSiblingDB('TweetScraper').getCollection('user').count());"

Export file

In [0]:
if db == 'mongo':
  !mongoexport -d TweetScraper -c tweet --type csv --fieldFile=fields.txt -o tweet.csv

## MySQL save to CSV (Optional)

Number of records

In [0]:
if db == 'mysql':
  !mysql TweetScraper -e "select COUNT(*) from scraper"

+----------+
| COUNT(*) |
+----------+
|    17024 |
+----------+


Export file

In [0]:
if db == 'mysql':
  !mysql TweetScraper -e "select * from scraper" -B | sed "s/'/\'/;s/\t/\",\"/g;s/^/\"/;s/$/\"/;s/\n//g" > tweet.csv

Export sql

In [0]:
if db == 'mysql':
  !mongodump TweetScraper scraper -o tweet.sql

2019-10-16T02:30:10.528+0000	positional arguments not allowed: [TweetScraper scraper]
2019-10-16T02:30:10.528+0000	try 'mongodump --help' for more information


# Download the data

In [0]:
filename = search.replace(' ', '_')
filename = filename.replace(',', '__')
!mv tweet.csv {filename}.csv

## Download to Drive

In [0]:
if store_in_drive:
  !cp tweet.csv /content/drive/My\ Drive/{filename}.csv

## Download to local

In [0]:
if download_file:
  import time
  from google.colab import files
  time.sleep(2)
  try:
    files.download(filename + '.csv')
  except:
    print('Please retry')